## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description,Date
0,0,Bluff,-46.6000,168.3333,53.80,76,100,10.92,NZ,overcast clouds,2022-01-25 23:44:06
1,1,Gat,31.6100,34.7642,50.40,100,9,10.38,IL,clear sky,2022-01-25 23:44:07
2,2,Yuli,9.6989,10.2744,69.87,18,79,10.38,NG,broken clouds,2022-01-25 23:44:07
3,3,Vaini,-21.2000,-175.2000,84.36,79,20,10.36,TO,few clouds,2022-01-25 23:41:06
4,4,Ruwi,23.5996,58.5507,62.29,65,0,5.86,OM,clear sky,2022-01-25 23:44:07


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 54
What is the maximum temperature you would like for your trip? 66


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count().isnull()

City_ID                False
City                   False
Lat                    False
Lng                    False
Max Temp               False
Humidity               False
Cloudiness             False
Wind Speed             False
Country                False
Current Description    False
Date                   False
dtype: bool

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.count().dropna()
clean_df = preferred_cities_df

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Ruwi,OM,62.29,clear sky,23.5996,58.5507,
15,Busselton,AU,63.09,clear sky,-33.6500,115.3333,
16,Comodoro Rivadavia,AR,60.66,clear sky,-45.8667,-67.5000,
18,New Norfolk,AU,64.65,overcast clouds,-42.7826,147.0587,
20,Hobart,AU,65.01,broken clouds,-42.8794,147.3294,
34,Shwebo,MM,59.14,few clouds,22.5667,95.7000,
39,Fortuna,US,60.19,clear sky,40.5982,-124.1573,
47,Rawson,AR,62.47,clear sky,-43.3002,-65.1023,
58,Jimma,ET,58.69,few clouds,7.6667,36.8333,
64,Mwingi,KE,63.21,clear sky,-0.9344,38.0601,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key  
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    #hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Ruwi,OM,62.29,clear sky,23.5996,58.5507,Al Falaj Hotel
15,Busselton,AU,63.09,clear sky,-33.6500,115.3333,Observatory Guest House
16,Comodoro Rivadavia,AR,60.66,clear sky,-45.8667,-67.5000,Austral Hotel
18,New Norfolk,AU,64.65,overcast clouds,-42.7826,147.0587,The Shingles Riverside Cottages
20,Hobart,AU,65.01,broken clouds,-42.8794,147.3294,Mantra on Collins Hobart


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.tail()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
698,Asfi,MA,57.87,overcast clouds,32.2994,-9.2372,Hôtel Abda
702,Nouadhibou,MR,65.48,broken clouds,20.9310,-17.0347,El Medina
705,Mpongwe,ZM,63.03,overcast clouds,-13.5091,28.1550,Casablanca guest house and bar
709,Alamogordo,US,56.66,broken clouds,32.8995,-105.9603,Fairfield Inn & Suites by Marriott Alamogordo
712,Marzuq,YE,57.04,clear sky,14.4000,46.4667,


In [11]:
clean_hotel_df["Hotel Name"].unique()

array(['Al Falaj Hotel', 'Observatory Guest House', 'Austral Hotel',
       'The Shingles Riverside Cottages', 'Mantra on Collins Hobart',
       'MOON SHINE Guest House', 'Super 8 by Wyndham Fortuna',
       'Hosteria Sampedro', 'Jimma Central Hotel',
       'Thorn Tree by The Address', 'Bredasdorp Country Manor',
       'Kaitangata Motor Camp', 'Hostal Las Lilas', 'Hotel do Campo', '',
       'Iberostar Founty Beach', 'Quinta da Meia Eira',
       'Hilltop, Hafazgona, Bandarban', 'Tengye Hotel', 'Ritz Lauca',
       'Kingsgate Hotel Te Anau', 'Shangcheng Hotel', 'Blue Bay Lodge',
       'محمد هادي ال فطيح', 'Hotel RIKMAN Continental', 'Complexe',
       'Shishime Hotel', 'Casa Laguna, Bed & Breakfast',
       'Hospitality Esperance', 'Eng Hiirey House', "Don Eddie's Landing",
       'Southern Beach Hotel & Resort Okinawa', 'Casa Das Faias',
       'Dibbo areaمنطقة دبو', 'Scenic Hotel Southern Cross',
       'Pines Inn & Suites', 'Hotel Arena Gruesa', 'Silk Suites',
       'Wingate by

In [12]:
clean_hotel_df.shape

(75, 7)

In [13]:
hotel_df.shape

(76, 7)

In [14]:
clean_hotel_df = clean_hotel_df[clean_hotel_df["Hotel Name"] != ""]

In [15]:
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Ruwi,OM,62.29,clear sky,23.5996,58.5507,Al Falaj Hotel
15,Busselton,AU,63.09,clear sky,-33.6500,115.3333,Observatory Guest House
16,Comodoro Rivadavia,AR,60.66,clear sky,-45.8667,-67.5000,Austral Hotel
18,New Norfolk,AU,64.65,overcast clouds,-42.7826,147.0587,The Shingles Riverside Cottages
20,Hobart,AU,65.01,broken clouds,-42.8794,147.3294,Mantra on Collins Hobart


In [16]:
clean_hotel_df.tail()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
696,Adra,ES,57.06,overcast clouds,36.7496,-3.0206,Hostal Zapata
698,Asfi,MA,57.87,overcast clouds,32.2994,-9.2372,Hôtel Abda
702,Nouadhibou,MR,65.48,broken clouds,20.9310,-17.0347,El Medina
705,Mpongwe,ZM,63.03,overcast clouds,-13.5091,28.1550,Casablanca guest house and bar
709,Alamogordo,US,56.66,broken clouds,32.8995,-105.9603,Fairfield Inn & Suites by Marriott Alamogordo


In [17]:
# 8a. Create the output File (CSV)
output_data_file =" WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [26]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} and {Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [28]:
# 11a. Add a marker layer for each city to the map.

max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))